In [1]:
!pip install boto3
!pip install --upgrade google-api-python-client

In [70]:
import boto3
import hashlib
import json
import os
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import yt_dlp
from datetime import datetime
from dotenv import load_dotenv


def current_timestamp():
    """Returns the current timestamp formatted for readability."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def get_env_variables():
    """Fetch all necessary configurations from environment variables."""
    return {
        'DEVELOPER_KEY': os.getenv('DEVELOPER_KEY'),
        'AWS_ACCESS_KEY_ID': os.getenv('AWS_ACCESS_KEY_ID'),
        'AWS_SECRET_ACCESS_KEY': os.getenv('AWS_SECRET_ACCESS_KEY'),
        'SEARCH_CACHE_TABLE': os.getenv('SEARCH_CACHE_TABLE'),
        'RESULTS_TABLE_NAME': os.getenv('RESULTS_TABLE_NAME'),
        'SEARCH_QUERY': os.getenv('SEARCH_QUERY'),
        'MAX_RESULTS': int(os.getenv('MAX_RESULTS', 3)),
        'ORDER': os.getenv('ORDER', 'viewCount'),
        'VIDEO_DURATION': os.getenv('VIDEO_DURATION', 'medium'),
        'PUBLISHED_AFTER': os.getenv('PUBLISHED_AFTER', '2010-01-01T00:00:00Z'),
        'PUBLISHED_BEFORE': os.getenv('PUBLISHED_BEFORE', '2024-12-31T23:59:59Z'),
        'RELEVANCE_LANGUAGE': os.getenv('RELEVANCE_LANGUAGE', 'en'),
        # 'VIDEO_CATEGORY_ID': os.getenv('VIDEO_CATEGORY_ID', '10'),
        'AWS_REGION': os.getenv('AWS_REGION')  # Add AWS region to the environment variables
    }

def open_aws_dynamodb_session(options):
    """
    Configures a boto3 session using AWS credentials read from environment variable.
    Returns a boto3 DynamoDB resource configured with these credentials.
    """
    try:

        # Configure the boto3 session with the read credentials
        session = boto3.Session(
            aws_access_key_id=options['AWS_ACCESS_KEY_ID'],
            aws_secret_access_key=options['AWS_SECRET_ACCESS_KEY'],
            region_name=options['AWS_REGION']  # Specify your AWS region
        )
        dynamodb_resource = session.resource('dynamodb')
        print(f"Successfully open_aws_dynamodb_session with environment variables. at {current_timestamp()}\n")
        # Return the configured DynamoDB resource
        return dynamodb_resource

    except Exception as e:
        print(f"Error open_aws_dynamodb_session from environment variables: {e} at {current_timestamp()}\n")
        return None


def cache_key(options):
    """Generate a hash key for the given search options."""
    key_str = json.dumps(options, sort_keys=True)
    return hashlib.sha256(key_str.encode('utf-8')).hexdigest()

def check_cache(options, dynamodb):
    """Check if search options are in the cache on AWS DynamoDB."""
    print(f"Checking cache at {current_timestamp()}\n")
    non_search_terms = ['DEVELOPER_KEY', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'SEARCH_CACHE_TABLE', 'RESULTS_TABLE_NAME', 'AWS_REGION', 'MAX_RESULTS']
    search_query_options = {k: v for k, v in options.items() if k not in non_search_terms}
    key = cache_key(search_query_options)
    sort_key_value = current_timestamp()

    try:
        cache_table = dynamodb.Table(options['SEARCH_CACHE_TABLE'])
        response = cache_table.get_item(
        Key={
                'CacheKey': key #,
                # 'Timestamp': sort_key_value  # You'll need to provide the appropriate sort key value here
            }
        )
        if 'Item' in response:
            print("Cache hit.")
            return json.loads(response['Item']['Results'])
    except Exception as e:
        print(f"Error accessing DynamoDB due to 'check_cache' malfunctioniong: {e} at {current_timestamp()}\n")
    print("Cache miss.\n")
    return None

def update_cache(options, dynamodb, results):
    """Update the cache with new search results, including the search time."""
    print(f"Updating cache at {current_timestamp()}\n")
    non_search_terms = ['DEVELOPER_KEY', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'SEARCH_CACHE_TABLE', 'RESULTS_TABLE_NAME', 'AWS_REGION', 'MAX_RESULTS']
    search_query_options = {k: v for k, v in options.items() if k not in non_search_terms}
    key = cache_key(search_query_options)
    timestamp = current_timestamp()  # Capture the search timestamp
    try:
        search_cache_table = options['SEARCH_CACHE_TABLE']
        cache_table = dynamodb.Table(search_cache_table)
        cache_table.put_item(Item={
            'CacheKey': key,
            'Results': json.dumps(results),
            'Timestamp': timestamp  # Store the search time
        })
    except Exception as e:
        print(f"Error updating DynamoDB due to 'update_cache' malfunctioniong: {e} at {current_timestamp()}\n")

def youtube_search_all_videos(options, dynamodb):
    """Perform a search on YouTube Data APIand return all videos based on the options, with caching and paging."""
    print(f"Performing YouTube search at {current_timestamp()}\n")
    cached_results = check_cache(options, dynamodb)
    if cached_results is not None:
        print(f"Function 'youtube_search_all_videos' was run at {current_timestamp()}\n")
        return cached_results

    developer_key = options['DEVELOPER_KEY']
    # CREATE YOUTUBE OBJECT calling the youtbue data api 
    youtube = build('youtube', 'v3', developerKey=developer_key)
    
    all_videos = []
    page_token = None
    max_iterations = 1  # Adjust based on how many pages you want to retrieve (50 results max with 25 per page)

    try:
        for _ in range(max_iterations):
            search_response = youtube.search().list(
                q=options['SEARCH_QUERY'],
                part='id,snippet',
                maxResults=options['MAX_RESULTS'],
                order=options['ORDER'],
                type='video',
                videoDuration=options['VIDEO_DURATION'],
                publishedAfter=options['PUBLISHED_AFTER'],
                publishedBefore=options['PUBLISHED_BEFORE'],
                relevanceLanguage=options['RELEVANCE_LANGUAGE'],
                # videoCategoryId=options['VIDEO_CATEGORY_ID'], # When there is a category ID assigned, if number 10, then results are mostly music.
                pageToken=page_token
            ).execute()

            all_videos.extend(search_response.get('items', []))
            page_token = search_response.get('nextPageToken')
            if not page_token:
                break

    except HttpError as e:
        print(f"An HTTP error occurred: {e.resp.status} {e.content} at {current_timestamp()}\n")
    
    update_cache(options, dynamodb, all_videos)
    print(f"Function 'youtube_search_all_videos' was run at {current_timestamp()}\n")
    return all_videos

def download_subtitles(video_id):
    """Download subtitles for a given YouTube video ID."""
    video_url = f'https://www.youtube.com/watch?v={video_id}'
    ydl_opts = {
        'writeautomaticsub': True,
        'subtitleslangs': ['en'],
        'skip_download': True,
        'outtmpl': f'subtitles/{video_id}.%(ext)s',
        'quiet': True
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
            subtitle_file = f'subtitles/{video_id}.en.vtt'
            if os.path.exists(subtitle_file):
                with open(subtitle_file, 'r', encoding='utf-8') as file:
                    subtitle_text = file.read()
                    print(f"Downloaded video subtitles for video ID: {video_id} into the file {subtitle_file} at {current_timestamp()}\n")
                return subtitle_file, subtitle_text
            return None, None
    except Exception as e:
        print(f"Failed to download subtitles for video ID: {video_id}: {e} at {current_timestamp()}\n")
        return None, None

def send_to_dynamodb(options, dynamodb, video_details):   
    """Send Youtube Data API results on the video and subtitle transcripts to Dynamodb."""
    try:
        results_table_name = options['RESULTS_TABLE_NAME']
        results_table = dynamodb.Table(results_table_name)
        response = results_table.put_item(Item=video_details)
        print(f"Successfully inserted api results and converted transcripts into DynamoDB  at {current_timestamp()}\n", response)
    except Exception as e:
        print(f"Error inserting into DynamoDB: {e}")

def process_videos_and_store(options, dynamodb, all_videos):
    """Process each video from the search results and store them."""
    for video in all_videos:
        video_id = video['id']['videoId']
        subtitle_file, subtitle_text = download_subtitles(video_id)
        video_details = {
            'VideoID': video_id,
            'Title': video['snippet']['title'],
            'URL': f'https://www.youtube.com/watch?v={video_id}',
            'PublishedAt': video['snippet']['publishedAt'],
            'SubtitleFile': subtitle_file if subtitle_file else "N/A",
            'SubtitleText': subtitle_text if subtitle_text else "N/A",
            # Including search parameters as part of video_details
            'SearchQuery': options['SEARCH_QUERY'],
            'Order': options['ORDER'],
            'VideoDuration': options['VIDEO_DURATION'],
            'RelevanceLanguage': options['RELEVANCE_LANGUAGE']
            # 'VideoCategoryID': options['VIDEO_CATEGORY_ID']
        }
        send_to_dynamodb(options, dynamodb, video_details)
        print(f"\nFunction 'process_videos_and_store' was run at {current_timestamp()} for video ID: {video_id}, title: {video['snippet']['title']}\n")

def main():

    # Load the .env file
    load_dotenv()

    # Ensure the subtitles directory exists}
    os.makedirs('subtitles', exist_ok=True)
    
    # Get environment variables
    options = get_env_variables()
    
    # Only proceed if options were successfully retrieved
    if options:
        dynamodb = open_aws_dynamodb_session(options)
        
        # Only proceed if DynamoDB was successfully configured
        if dynamodb:

            all_videos = youtube_search_all_videos(options, dynamodb)

            # Only proceed if videos were successfully retrieved
            if all_videos:
                process_videos_and_store(options, dynamodb, all_videos)
            else:
                print(f"Failed to retrieve videos at {current_timestamp()}. Exiting...\n")
        else:
            print(f"Failed to configure boto3 with Docker secrets at {current_timestamp()}. Exiting...\n")
    else:
        print(f"Failed to retrieve environment variables at {current_timestamp()} Exiting...\n")

if __name__ == "__main__":
    main()


Successfully open_aws_dynamodb_session with environment variables. at 2024-02-22 00:30:04

Performing YouTube search at 2024-02-22 00:30:04

Checking cache at 2024-02-22 00:30:04

Error accessing DynamoDB due to 'check_cache' malfunctioniong: An error occurred (ValidationException) when calling the GetItem operation: The provided key element does not match the schema at 2024-02-22 00:30:04

Cache miss.

Updating cache at 2024-02-22 00:30:04

Function 'youtube_search_all_videos' was run at 2024-02-22 00:30:05

Downloaded video subtitles for video ID: fG1oNm2tCro into the file subtitles/fG1oNm2tCro.en.vtt at 2024-02-22 00:30:07

Successfully inserted api results and converted transcripts into DynamoDB  at 2024-02-22 00:30:07
 {'ResponseMetadata': {'RequestId': '6R1DP4GQ7STO56RN1NUQL98T6BVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Thu, 22 Feb 2024 08:30:07 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '2', 'conne

In [59]:
import boto3

def print_dynamodb_table_items(table_name, aws_region='us-east-2'):
    """
    Prints all items stored in a specified DynamoDB table.

    :param table_name: The name of the DynamoDB table to print items from.
    :param aws_region: The AWS region where the table is located. Default is 'us-east-1'.
    """
    # Initialize a DynamoDB service resource in the specified region
    dynamodb = boto3.resource('dynamodb', region_name=aws_region)
    
    # Connect to the table
    table = dynamodb.Table(table_name)
    
    # Scan the table - note that this can be costly for tables with many items
    try:
        response = table.scan()
        
        # Check if items were found
        if 'CacheKey' in response:
            items = response['CacheKey']
            print(f"Items in {table_name}:")
            for item in items:
                print(item)
        else:
            print(f"No items found in {table_name}.")
            
        # Handling potential pagination
        while 'LastEvaluatedKey' in response:
            response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
            items = response['CacheKey']
            for item in items:
                print(item)
                
    except Exception as e:
        print(f"Error accessing DynamoDB: {e}")

# Example usage
table_name = 'YTSearchCache'
print_dynamodb_table_items(table_name)


No items found in YTSearchCache.


In [58]:
import boto3
import json

def print_ytsearchcache_items(aws_region='us-east-2'):
    """
    Prints all items stored in the 'YTSearchCache' DynamoDB table, deserializing
    the JSON stored in the 'Results' field for readability.

    :param aws_region: The AWS region where the table is located. Default is 'us-east-1'.
    """
    table_name = 'YTSearchCache'
    
    # Initialize a DynamoDB service resource in the specified region
    dynamodb = boto3.resource('dynamodb', region_name=aws_region)
    
    # Connect to the YTSearchCache table
    table = dynamodb.Table(table_name)
    
    # Scan the table - note that this can be costly for tables with many items
    try:
        response = table.scan()
        
        # Check if items were found
        if 'Items' in response:
            print(f"Items in {table_name}:")
            for item in response['Items']:
                print(f"CacheKey: {item['CacheKey']}")
                print(f"Timestamp: {item['Timestamp']}")
                
                # Deserialize the JSON in the 'Results' field
                results = json.loads(item['Results'])
                print("Results:")
                print(json.dumps(results, indent=4))  # Pretty-print the results
                
        else:
            print(f"No items found in {table_name}.")
            
        # Handling potential pagination
        while 'LastEvaluatedKey' in response:
            response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
            for item in response['Items']:
                print(f"CacheKey: {item['CacheKey']}")
                print(f"Timestamp: {item['Timestamp']}")
                results = json.loads(item['Results'])
                print("Results:")
                print(json.dumps(results, indent=4))
                
    except Exception as e:
        print(f"Error accessing DynamoDB: {e}")

# Example usage
print_ytsearchcache_items()


Items in YTSearchCache:
